### Meta-Learners Examples - Single/Multiple Treatment Cases

[传送门](https://causalml.readthedocs.io/en/latest/examples/meta_learners_with_synthetic_data_multiple_treatment.html)

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from xgboost import XGBRegressor, XGBClassifier
import warnings

# from causalml.inference.meta import XGBTLearner, MLPTLearner
from causalml.inference.meta import BaseSRegressor, BaseTRegressor, BaseXRegressor, BaseRRegressor
from causalml.inference.meta import BaseSClassifier, BaseTClassifier, BaseXClassifier, BaseRClassifier
from causalml.inference.meta import LRSRegressor
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.dataset import *
from causalml.metrics import *

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# import from package
import logging
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import statsmodels.api as sm
from copy import deepcopy

logger = logging.getLogger('causalml')
logging.basicConfig(level=logging.INFO)

%matplotlib inline

### Single Treatment Case

In [6]:
# Generate synthetic data using mode 1
y, X, treatment, tau, b, e = synthetic_data(mode=1, n=10000, p=8, sigma=1.0)

print(treatment)

treatment = np.array(['treatment_a' if val == 1 else 'control' for val in treatment])
print(treatment)

[1 1 0 ... 1 0 1]
['treatment_a' 'treatment_a' 'control' ... 'treatment_a' 'control'
 'treatment_a']


In [7]:
# S-Learner ATE
learner_s = BaseSRegressor(XGBRegressor(), control_name='control')
ate_s = learner_s.estimate_ate(X=X, treatment=treatment, y=y, return_ci=False, bootstrap_ci=False)

INFO:causalml:Error metrics for group treatment_a
INFO:causalml:    RMSE   (Control):     0.6623
INFO:causalml:    RMSE (Treatment):     0.6813
INFO:causalml:   sMAPE   (Control):     0.6468
INFO:causalml:   sMAPE (Treatment):     0.3733
INFO:causalml:    Gini   (Control):     0.8211
INFO:causalml:    Gini (Treatment):     0.8161


In [8]:
ate_s

array([0.51212267])

In [9]:
# S-Learner ATE w/Confidence Intervals
alpha = 0.05
learner_s = BaseSRegressor(XGBRegressor(), ate_alpha=alpha, control_name='control')
ate_s, ate_s_lb, ate_s_ub = learner_s.estimate_ate(X=X, treatment=treatment, y=y, return_ci=True, bootstrap_ci=True)

INFO:causalml:Error metrics for group treatment_a
INFO:causalml:    RMSE   (Control):     0.6623
INFO:causalml:    RMSE (Treatment):     0.6813
INFO:causalml:   sMAPE   (Control):     0.6468
INFO:causalml:   sMAPE (Treatment):     0.3733
INFO:causalml:    Gini   (Control):     0.8211
INFO:causalml:    Gini (Treatment):     0.8161
INFO:causalml:Bootstrap Confidence Intervals for ATE
100%|██████████| 1000/1000 [22:39<00:00,  1.36s/it]


In [ ]:
np.vstack((ate_s_lb, ate_s, ate_s_ub))

In [ ]:
# S-Learner ATE w/Boostrap Confidence Intervals
ate_s_b, ate_s_lb_b, ate_s_ub_b = learner_s.estimate_ate(X=X, treatment=treatment, y=y, return_ci=True,
                                                         bootstrap_ci=True, n_bootstraps=100, bootstrap_size=5000)

In [ ]:
np.vstack((ate_s_lb_b, ate_s_b, ate_s_ub_b))

In [ ]:
# S-Learner CATE
learner_s = BaseSRegressor(XGBRegressor(), control_name='control')
cate_s = learner_s.fit_predict(X=X, treatment=treatment, y=y, return_ci=False)

In [ ]:
cate_s

In [ ]:
# S-Learner CATE w/Confidence Intervals
alpha = 0.05
learner_s = BaseSRegressor(XGBRegressor(), ate_alpha=alpha, control_name='control')
cate_s, cate_s_lb, cate_s_ub = learner_s.fit_predict(X=X, treatment=treatment, y=y, return_ci=True,
                                                     n_bootstraps=100, bootstrap_size=5000)

In [ ]:
cate_s

In [ ]:
cate_s_lb

In [ ]:
cate_s_ub

In [ ]:
# T-Learner ATE w/Confidence Intervals
learner_t = BaseTRegressor(XGBRegressor(), control_name='control')
ate_t, ate_t_lb, ate_t_ub = learner_t.estimate_ate(X=X, treatment=treatment, y=y)

In [ ]:
np.vstack((ate_t_lb, ate_t, ate_t_ub))

In [ ]:
# T-Learner ATE w/Boostrap Confidence Intervals
ate_t_b, ate_t_lb_b, ate_t_ub_b = learner_t.estimate_ate(X=X, treatment=treatment, y=y, bootstrap_ci=True,
                                                         n_bootstraps=100, bootstrap_size=5000)

In [ ]:
np.vstack((ate_t_lb_b, ate_t_b, ate_t_ub_b))

In [ ]:
# T-Learner CATE
learner_t = BaseTRegressor(XGBRegressor(), control_name='control')
cate_t = learner_t.fit_predict(X=X, treatment=treatment, y=y)

In [ ]:
cate_t

In [ ]:
# T-Learner CATE w/Confidence Intervals
learner_t = BaseTRegressor(XGBRegressor(), control_name='control')
cate_t, cate_t_lb, cate_t_ub = learner_t.fit_predict(X=X, treatment=treatment, y=y, return_ci=True, n_bootstrap=100,
                                                     bootstrap_size=5000)

In [ ]:
cate_t

In [ ]:
cate_t_lb

In [ ]:
cate_t_ub

In [ ]:
# X-Learner ATE w/Confidence Intervals With Propensity Score Input
learner_x = BaseXRegressor(XGBRegressor(), control_name='control')
ate_x, ate_x_lb, ate_x_ub = learner_x.estimate_ate(X=X, treatment=treatment, y=y, p=e)

In [ ]:
np.vstack((ate_x_lb, ate_x, ate_x_ub))

In [ ]:
# X-Learner ATE w/Confidence Intervals Without Propensity Score input
ate_x_no_p, ate_x_lb_no_p, ate_x_ub_no_p = learner_x.estimate_ate(X=X, treatment=treatment, y=y)

In [ ]:
np.vstack((ate_x_lb_no_p, ate_x_no_p, ate_x_ub_no_p))

In [ ]:
learner_x.propensity_model